# Hybrid Recommendation on MathOverflow

To run this notebook, you'll need to download (this MathOverflow dump)[https://s3-eu-west-1.amazonaws.com/lateral-datadumps/mathoverflow.tar.gz] (56MB, compressed).  We assumed that you've de

In [1]:
import scipy.sparse as sp
import numpy as np
import random
from featuriser import Featuriser, TextAnalyser, TagAnalyser
from indexed_sparse_matrix import IndexedSparseMatrix as ISM
from fm import FM, CYTHON_FLOAT, CYTHON_UINT, EntityModel

## Hyperparameters

In [2]:
dimension = 50
margin = 0.85
learning_rate = 1
max_sampled = 50
min_interaction_count = 4
min_document_frequency = 5
feature_idf = True
epochs = 5
number_threads = 2

## Helper functions

In [3]:
def hstack_ism(rows, isms):
    """
    A wrapper for scipy's hstack to avoid the blocks=[] corner case.
    `ism` is an iterable of IndexedSparseMatrix instances.
    `rows` is an iterable of row ids.
    Returns an IndexedSparseMatrix (CSR) with dtype CYTHON_FLOAT, and a list of
    pairs defining the ranges of the input matrices in the stacked one.
    Syncs the row indices of each of the IndexedSparseMatrix with `rows` before
    stacking.  Column names are combined with the 0-offset index of their
    original IndexedSparseMatrix in a tuple e.g. (2, "banana").
    """
    if len(isms):
        blocks = []
        cols = []
        for i, ism in enumerate(isms):
            ism.sync_row_index(rows)
            blocks.append(ism.M)
            cols.extend([(i, col_id) for col_id in ism.cols])
        ends = np.cumsum([b.shape[1] for b in blocks]).tolist()
        starts = [0] + ends[:-1]
        ranges = list(zip(starts, ends))
        csr_mat = sp.hstack(blocks, format='csr', dtype=CYTHON_FLOAT)
        return ISM(csr_mat, rows=rows, cols=cols), ranges
    else:
        csr_mat = sp.csr_matrix((len(rows), 0), dtype=CYTHON_FLOAT)
        return ISM(csr_mat, rows=rows, cols=[]), []

    
def extract_test_interactions(interactions, N):
    """
    Given a 0/1 valued COO-matrix interactions, create and return a new
    COO-matrix containing N of the non-zero entries, randomly chosen.  These
    values are subtracted from `interactions`.
    Return dtypes and shapes are those of `interactions`.
    """
    indices = random.sample(range(len(interactions.M.data)), N)
    for index in indices:
        interactions.M.data[index] = 0
    test_data = np.ones(len(indices), dtype=interactions.M.dtype)
    test_row = [interactions.M.row[i] for i in indices]
    test_col = [interactions.M.col[i] for i in indices]
    test_coo = sp.coo_matrix((test_data, (test_row, test_col)),
                             dtype=interactions.M.dtype,
                             shape=interactions.M.shape)
    return ISM(test_coo, rows=interactions.rows, cols=interactions.cols)


def count_occurrences_ism(ism, axis):
    """
    Given a IndexedSparseMatrix instance wrapping a COO matrix, return a
    dictionary mapping each id on the specified axis to the number of nonzero
    entries in its row/column.
    """
    ids = [ism.rows, ism.cols][axis]
    counts = count_occurrences_coo(ism.M, axis)
    return dict(zip(ids, counts))


def count_occurrences_coo(coo, axis):
    """
    Given a COO matrix, return a 1d numpy array mapping each index on the
    specified axis to the number of nonzero entries in its row/column.
    Dtype of returned array is np.int32.
    """
    length = coo.shape[axis]
    indices = [coo.row, coo.col][axis]
    weights = (coo.data != 0)
    counts = np.bincount(indices, weights, minlength=length).astype(np.int32)
    return counts


def featurise_ids(all_ids, feature_ids):
    """
    Given lists of distinct ids `all_ids` and `feature_ids`, where
    `feature_ids` is a subset of `all_ids`, return a IndexedSparseMatrix (CSR)
    of dtype CYTHON_FLOAT.
    Rows are enumerated by `all_ids`, columns by `feature_ids`, where all
    entries are 0/1, and 1 iff the id of the row is in `feature_ids`.
    """
    feature_mapper = dict(zip(feature_ids, range(len(feature_ids))))
    shape = (len(feature_ids),)
    col_indices = np.empty(shape, dtype=CYTHON_UINT)
    row_indices = np.empty(shape, dtype=CYTHON_UINT)
    data = np.ones(shape, dtype=CYTHON_FLOAT)
    data_no = 0  # number of the COO tuples have we specified so far
    for row_no, _id in enumerate(all_ids):
        if _id in feature_mapper:
            row_indices[data_no] = row_no
            col_indices[data_no] = feature_mapper[_id]  # i.e. the column number
            data_no += 1
    shape = (len(all_ids), len(feature_ids))
    csr_mat = sp.csr_matrix((data, (row_indices, col_indices)), shape=shape)
    return ISM(csr_mat, rows=all_ids, cols=feature_ids)
               
               
def report_percentile_rank(fm, test_interactions):
    print('calculating mean percentile rank')
    prs = fm.percentile_ranks(test_interactions.M.row,
                              test_interactions.M.col)
    stats = (np.mean([pr.pr for pr in prs]),
             len(prs),
             len(test_interactions.M.row))
    print('MPR=%.5f from %i / %i recognisable test interactions' % stats)

## Load interaction data

In [4]:
from lip.tools.indexed_sparse_matrix import IndexedSparseMatrix,\
    value_ordered_keys
from collections import defaultdict

def build_interaction_matrix(pairs):
    """
    `pairs` is a list of (user_id, document_id) pairs, representing interactions.
    """
    print('building interaction matrix')

    # define an enumeration of the ids
    user_mapper = defaultdict(lambda: (len(user_mapper), 0))
    document_mapper = defaultdict(lambda: (len(document_mapper), 0))
    
    n_interactions = len(pairs)
    shape = (n_interactions,)
    row_indices = np.empty(shape, dtype=np.int32)
    col_indices = np.empty(shape, dtype=np.int32)
       
    for i, (user_id, document_id) in enumerate(pairs):
        row_indices[i] = user_mapper[user_id][0]
        col_indices[i] = document_mapper[document_id][0]
        
    counts = (len(user_mapper), len(document_mapper), n_interactions)
    print('%i users, %i documents, %i preferences' % counts)

    user_ids = value_ordered_keys(user_mapper)
    document_ids = value_ordered_keys(document_mapper)
    data = np.ones(n_interactions, dtype=CYTHON_FLOAT)  # interactions data
    shape = (len(user_mapper), len(document_mapper))
    interactions = IndexedSparseMatrix(
        sp.coo_matrix((data, (row_indices, col_indices)),
                      shape=shape, dtype=CYTHON_FLOAT),
        rows=user_ids, cols=document_ids)
    return interactions

In [5]:
def load_interactions():
    pairs = []
    for line in open('dataset/interactions.csv'):
        bits = line.strip().split(',')
        pairs.append((bits[1], bits[0]))
    return pairs

interactions = build_interaction_matrix(load_interactions())

building interaction matrix
20490 users, 53591 documents, 613277 preferences


## Train/test split

In [6]:
test_interactions = extract_test_interactions(interactions, N=10000)

## Document feature selection

Note: we are ignoring documents for which we have no interaction data.

In [7]:
import csv

doc_ids_seen_counts = count_occurrences_ism(interactions, axis=1)

def id_text_iter():
    with open('dataset/docs-text.csv') as f:
        reader = csv.reader(f, delimiter=',', quotechar='"')
        for row in reader:
            if row[0] not in doc_ids_seen_counts:
                continue
            yield row
            
def id_tag_iter():
    with open('dataset/docs-tags.csv') as f:
        reader = csv.reader(f, delimiter=',', quotechar='"')
        for row in reader:
            if row[0] not in doc_ids_seen_counts:
                continue
            yield row

In [8]:
print('selecting document id features')
document_id_features = [_id for _id in interactions.cols
                        if doc_ids_seen_counts[_id] > min_interaction_count]

vocab_params = {'min_interaction_count': min_interaction_count,
                'min_document_frequency': min_document_frequency,
                'feature_idf': feature_idf,
                'dtype': CYTHON_FLOAT}

print('selecting text features')
text_featuriser = Featuriser(analyzer=TextAnalyser(), **vocab_params)
text_featuriser.fit(id_text_iter(), doc_ids_seen_counts)
print('%i word features' % len(text_featuriser.vocab))

selecting document id features
selecting text features
31517 word features


In [9]:
print('selecting tag features')
tag_featuriser = Featuriser(analyzer=TagAnalyser(), **vocab_params)
tag_featuriser.fit(id_tag_iter(), doc_ids_seen_counts)
print('%i tag features' % len(tag_featuriser.vocab))

selecting tag features
1052 tag features


## User feature selection

In [10]:
print('selecting user id features')
seen_counts = count_occurrences_ism(interactions, axis=0)
user_id_features = [_id for _id in interactions.rows
                    if seen_counts[_id] > min_interaction_count]
print('%i user id features' % len(user_id_features))

selecting user id features
8597 user id features


## Featurisation

In [11]:
print('featurising user ids')
user_feature_counts = featurise_ids(interactions.rows, user_id_features)

def featurise_documents(interactions, id_text_iter, id_tag_iter):
    """
    Build and return the document x document features CSR matrix
    (IndexedSparseMatrix) along with a dict mapping each feature type
    to a range (start, stop) that specifies which document features are of
    this type.
    """
    blocks = {}
    print('featurising document ids')
    ids = interactions.cols
    blocks['ids'] = featurise_ids(ids, document_id_features)
    print('featurising text')
    blocks['words'] = text_featuriser.transform(id_text_iter)
    print('featurising tags')
    blocks['tags'] = tag_featuriser.transform(id_tag_iter)
    print('joining document feature counts')
    feature_types, counts = zip(*blocks.items())
    joined_counts, ranges = hstack_ism(ids, counts)
    document_feature_ranges = dict(zip(feature_types, ranges))
    return joined_counts, document_feature_ranges

document_feature_counts, document_feature_ranges = featurise_documents(
    interactions, id_text_iter(), id_tag_iter())

featurising user ids
featurising document ids
featurising text
featurising tags
joining document feature counts


## Train the factorisation machine

In [12]:
users = EntityModel(dimension, learning_rate, user_feature_counts.M)
items = EntityModel(dimension, learning_rate, document_feature_counts.M)
fm =  FM(users, items, margin=margin, max_sampled=max_sampled)

print('fitting FM model')
for epoch in range(epochs):
    print('start epoch %s' % epoch)
    fm.learn(interactions.M, number_threads=number_threads)
    report_percentile_rank(fm, test_interactions)

fitting FM model
start epoch 0
calculating mean percentile rank
MPR=0.04396 from 9387 / 10000 recognisable test interactions
start epoch 1
calculating mean percentile rank
MPR=0.03588 from 9387 / 10000 recognisable test interactions
start epoch 2
calculating mean percentile rank
MPR=0.03341 from 9387 / 10000 recognisable test interactions
start epoch 3
calculating mean percentile rank
MPR=0.03238 from 9387 / 10000 recognisable test interactions
start epoch 4
calculating mean percentile rank
MPR=0.03153 from 9387 / 10000 recognisable test interactions
